# Env Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
! pip install scikit-surprise

     |████████████████████████████████| 3.3MB 5.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


# CollaboratIve Filtering

In [0]:
# Load Libraries
import numpy as np
import pandas as pd
import random
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

In [0]:
# Load Data
dir = '/content/drive/My Drive/DataScienceWiz/steam_webscraping/'
df = pd.read_csv(dir + 'new_user.csv',index_col = 0)

In [0]:
user = pd.read_csv(dir + 'user.csv',index_col = 0)

In [0]:
mu_user = user.groupby("appid").transform('mean')
st_user = user.groupby("appid").transform('std')
user["normalize"] = (user["playtime_forever"] - mu_user["playtime_forever"]) / st_user["playtime_forever"]

In [12]:
user.drop(['playtime_forever'], axis = 1, inplace = True)
user.head()

,userID,appid,normalize
0,76561198158086086,4000,-0.423320
1,76561198158086086,110800,-0.621365
2,76561198158086086,226320,-0.186106
3,76561198158086086,250320,0.064789
4,76561198158086086,296470,-0.371237


In [0]:
user.dropna(inplace = True)

In [6]:
df.drop(['playtime_forever'],axis = 1, inplace = True)
df.head()

,userID,appid,rating
0,76561198175177483,10,0.268908
1,76561198230470504,10,0.268908
2,76561198092770800,10,0.268908
3,76561198072618880,10,0.268908
4,76561197967251861,10,0.268908


In [0]:
reader = Reader(rating_scale=(1, df.shape[0]))
data = Dataset.load_from_df(df[['userID', 'appid', 'rating']], reader)

In [0]:
reader2 = Reader(rating_scale=(1,user.shape[0]))
data2 = Dataset.load_from_df(user[['userID','appid','normalize']],reader2)

## Baseline KNN

In [0]:
# Baseline Rec_Sys
trainset, testset = train_test_split(data, test_size=.25)
algo = KNNBasic()
pred_KNN = algo.fit(trainset).test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [0]:
accuracy.rmse(pred_KNN)

RMSE: 0.4843


0.48433646557821425

In [27]:
trainset, testset = train_test_split(data2, test_size=.25)
algo = KNNBasic()
pred_KNN_norm = algo.fit(trainset).test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [28]:
accuracy.rmse(pred_KNN_norm)

RMSE: 1.4130


1.413020836678079

## Tuning Parameters

In [0]:
# KNNBasic Tuning
knn_grid = {'k':[20,40,60,80,100],
            'min_k':[1,2]}

knn_grid_search = GridSearch(KNNBasic, knn_grid, measures=['RMSE','FCP'])
data.split(n_folds = 4)

knn_grid_search.evaluate(data)

/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:232: UserWarning: The GridSearch() class is deprecated. Please use model_selection.GridSearchCV instead.
  'model_selection.GridSearchCV instead.', UserWarning)


Running grid search for the following parameter combinations:
{'k': 20, 'min_k': 1}
{'k': 20, 'min_k': 2}
{'k': 40, 'min_k': 1}
{'k': 40, 'min_k': 2}
{'k': 60, 'min_k': 1}
{'k': 60, 'min_k': 2}
{'k': 80, 'min_k': 1}
{'k': 80, 'min_k': 2}
{'k': 100, 'min_k': 1}
{'k': 100, 'min_k': 2}


/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/lib/python3.6/dist-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [0]:
# Funk_SVD Tuning
from surprise import GridSearch

param_grid = {'n_epochs':[5, 10], 'lr_all':[0.002, 0.005],
             'reg_all':[0.4, 0.6]}

grid_search = GridSearch(SVD, param_grid, measures=['RMSE', 'FCP'])
data.split(n_folds=4)

grid_search.evaluate(data)

/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:232: UserWarning: The GridSearch() class is deprecated. Please use model_selection.GridSearchCV instead.
  'model_selection.GridSearchCV instead.', UserWarning)


Running grid search for the following parameter combinations:
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}


/usr/local/lib/python3.6/dist-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/usr/local/lib/python3.6/dist-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Resulsts:
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}
{'RMSE': 0.4820249001885544, 'FCP': 0.060966644770487154}
----------
{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}
{'RMSE': 0.4820248596456199, 'FCP': 0.0}
----------
{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}
{'RMSE': 0.48202508628643775, 'FCP': 0.07459802433032205}
----------
{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}
{'RMSE': 0.4820248690976233, 'FCP': 0.03382225606507033}
----------
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}
{'RMSE': 0.48202500284739064, 'FCP': 0.08156412642147493}
----------
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}
{'RMSE': 0.48202486405853884, 'FCP': 0.03382225606507033}
----------
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
{'RMSE': 0.48202519002113764, 'FCP': 0.07307127294945276}
----------
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}
{'RMSE': 0.48202487205109085, 'FCP': 0.04606122266218023}
----------


In [0]:
#FCP score: Fraction of Concordant Pairs
print(grid_search.best_score['FCP'])
print(grid_search.best_params['FCP'])

0.08156412642147493
{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}


## Baseline SVD

In [0]:
# Baseline Rec_Sys
trainset, testset = train_test_split(data, test_size=.25)
algo = SVD()
pred_SVD = algo.fit(trainset).test(testset)

In [0]:
accuracy.rmse(pred_SVD)

RMSE: 0.4825


0.4824900125847704